In [1]:
import pandas as pd
import difflib
import datetime
import dateutil
import xgboost
import category_encoders as ce
import scipy.stats as stats
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
def DEDUP_listingid(df):
    
    df['listing_id']    = df['listing_id'].astype(int).astype(str).str.strip()
    df                  = df.drop_duplicates('listing_id')

    return (df)
    
def FIXNULL_make(df): 

    df['make']          = df['make'].str.upper()
    df['make']          = df['make'].str.replace(' ','').str.strip()
    df['title']         = df['title'].str.upper()
    
    make_list           = [make for make in df['make'].unique().tolist() if type(make) == str]
    df['make_temp']     = df['title'].str.split(' ').str[0]
    df['make_temp']     = df.apply(lambda x: difflib.get_close_matches(x['make_temp'], make_list, n=1)[0], axis=1)
    df['make']          = df['make'].fillna(df['make_temp'])
    df                  = df.drop(columns = 'make_temp')

    return (df)

def FIXNULL_manufacture(df):

    df['original_reg_date']         = pd.to_datetime(df['original_reg_date'], format = "%d-%b-%Y")
    df['reg_date']                  = pd.to_datetime(df['reg_date'], format = "%d-%b-%Y")
    
    df['original_reg_date_temp']    = df['original_reg_date'].dt.year
    df['reg_date_temp']             = df['reg_date'].dt.year
    df['manufactured']              = df['manufactured'].fillna(df[['original_reg_date_temp','reg_date_temp']].min(axis=1))
    df['manufactured']              = df['manufactured'].astype(int).astype(str)
    df                              = df.drop(columns = ['original_reg_date_temp', 'reg_date_temp'])
    
    return (df)

def FIXNULL_ownercount(df):
    
    df['no_of_owners'] = df['no_of_owners'].fillna(1)
    
    return df

def FIXNULL_dereg(df):
    df['dereg_value'] = df['dereg_value'].fillna(0)
    return(df)

def ENCODE_category(df, drop = True):
    
    df_temp             = df['category'].str.split(',', expand = True)
    df_temp             = df_temp.astype(str)
    category_set        = set(sum(df['category'].str.split(',').tolist(), []))
    category_set        = set([category.strip() for category in category_set if len(category.strip()) > 1])
    category_dflist     = [df_temp.apply(lambda x: sum(x.str.contains(category)), axis=1).rename(f"CAT-{category.replace(' ','')}") for category in category_set]
    df                  = pd.concat([df]+category_dflist, axis = 1)
    df                  = df.drop(columns = ['category'])

    df['CAT-coeparf'] = df['CAT-coecar'].astype(str).str.replace('1','COE').str.replace('0', 'PARF')
    df['CAT-elechybrid'] = df['CAT-hybridcars'].astype(str).str.replace('1','HYBRID').str.replace('0', 'ELEC')

    if drop == True:
        df = df.drop(columns = ['CAT-electriccars','CAT-hybridcars','CAT-coecar', 'CAT-parfcar',])
        
    return (df)

def ENCODE_transmission(df, drop=True):
    
    df['TRANSMISSION-manual'] = pd.get_dummies(df['transmission'], drop_first=True, dtype=int)
    
    if drop == True:
        df              = df.drop(columns = ['transmission'])
    
    return (df)

def ENCODE_vehtype(df, drop=True):

    df['type_of_vehicle'] = df['type_of_vehicle'].str.upper().str.strip().str.replace(' ','')
    encoder             = ce.BinaryEncoder(cols='type_of_vehicle',return_df=True)
    df_temp             = encoder.fit_transform(df['type_of_vehicle']) 
    df_temp.columns     = [col.replace('type_of_vehicle_', "TYPE-binenc") for col in df_temp.columns] 

    df                  = pd.concat([df, df_temp], axis = 1)

    if drop == True:
        df                  = df.drop(columns = 'type_of_vehicle')

    return df

def ENCODE_make(df, drop=True):

    encoder             = ce.BinaryEncoder(cols='make',return_df=True)
    df_temp             = encoder.fit_transform(df['make']) 
    df_temp.columns     = [col.replace('make_', "MAKE-binenc") for col in df_temp.columns] 

    df = pd.concat([df, df_temp], axis =1)

    if drop==True:
        df = df.drop(columns = ['make'])
    
    return(df)

def ENCODE_opcscheme(df): 
    df.loc[~(df['opc_scheme'].isnull()),'opc_scheme'] = 1
    df['opc_scheme'] = df['opc_scheme'].fillna(0)

    return df

def ENCODE_fueltype(df, drop=True):

    df['fuel_type'] = df['fuel_type'].fillna('unknown').str.upper().str.strip()
    encoder         = ce.BinaryEncoder(cols='fuel_type',return_df=True)
    df_temp         = encoder.fit_transform(df['fuel_type']) 
    df_temp.columns = [col.replace('fuel_type_', "FUEL-binenc") for col in df_temp.columns] 

    df = pd.concat([df, df_temp], axis = 1)

    if drop==True:
        df = df.drop(columns = 'fuel_type')

    return df

def ENCODE_model(df, drop=True):

    df['model']     = df['model'].str.upper()
    
    encoder         = ce.BinaryEncoder(cols='model',return_df=True)
    df_temp         = encoder.fit_transform(df['model']) 
    df_temp.columns = [col.replace('model_', "MODEL-binenc") for col in df_temp.columns] 
    df              = pd.concat([df, df_temp], axis = 1)

    if drop==True:
        df = df.drop(columns = 'model')

    return df

def EXTRACT_features(df):

    df['features'] = df['features'].astype(str).str.upper()
    
    df.loc[(df['features'].str.contains('4 CYLINDER')) | (df['features'].str.contains('4-CYLINDER')) | (df['features'].str.contains('FOUR CYLINDER')), 'FEAT-CYLcount'] = 4
    df.loc[(df['features'].str.contains('5 CYLINDER')) | (df['features'].str.contains('5-CYLINDER')) | (df['features'].str.contains('FIVE CYLINDER')), 'FEAT-CYLcount'] = 5
    df.loc[(df['features'].str.contains('6 CYLINDER')) | (df['features'].str.contains('6-CYLINDER')) | (df['features'].str.contains('SIX CYLINDER')), 'FEAT-CYLcount'] = 6
    df['FEAT-CYLcount'] = df['FEAT-CYLcount'].fillna(0).astype(int)
    
    df['FEAT-brakehp'] = df['features'].str.extract('(\d+)(?:BHP| BHP)') 
    df['FEAT-torque'] = df['features'].str.extract('(\d+)(?:NM| NM)') 
    df['FEAT-valve'] = df['features'].str.extract('(\d+)(?:VALVE| VALVE)') 

    df.loc[(df['features'].str.contains('TURBOCHARGED')) | (df['features'].str.contains('SUPERCHARGED')), 'FEAT-charged_engine'] = 1
    df['FEAT-charged_engine'] = df['FEAT-charged_engine'].fillna(0).astype(int)
    
    df.loc[df['features'].str.contains('SRS'), 'FEAT-srs_airbag'] = 1
    df['FEAT-srs_airbag'] = df['FEAT-srs_airbag'].fillna(0).astype(int)
    df.loc[df['features'].str.contains('KEYLESS'), 'FEAT-keyless_ss'] = 1
    df['FEAT-keyless_ss'] = df['FEAT-keyless_ss'].fillna(0).astype(int)
    df.loc[df['features'].str.contains('GUARANTEE'), 'FEAT-guarantee'] = 1
    df['FEAT-guarantee'] = df['FEAT-guarantee'].fillna(0).astype(int)
    
    df.loc[df['features'].str.contains('AWD'), 'FEAT-awd'] = 1
    df['FEAT-awd'] = df['FEAT-awd'].fillna(0).astype(int)
    df.loc[df['features'].str.contains('4WD'), 'FEAT-4wd'] = 1
    df['FEAT-4wd'] = df['FEAT-4wd'].fillna(0).astype(int)
    df.loc[df['features'].str.contains('RWD'), 'FEAT-rwd'] = 1
    df['FEAT-rwd'] = df['FEAT-rwd'].fillna(0).astype(int)
    df.loc[df['features'].str.contains('FWD'), 'FEAT-fwd'] = 1
    df['FEAT-fwd'] = df['FEAT-fwd'].fillna(0).astype(int)
    
    df = df.drop(columns = 'features')
    
    return df

def EXTRACT_desc(df): 
    df['description'] = df['description'].astype(str).str.upper()

    df.loc[df['description'].str.contains('LOAN'), 'DESC-loan'] = 1
    df['DESC-loan'] = df['DESC-loan'].fillna(0).astype(int)

    df.loc[df['description'].str.contains('MAINTAINED'), 'DESC-maintained'] = 1
    df['DESC-maintained'] = df['DESC-maintained'].fillna(0).astype(int)

    df.loc[df['description'].str.contains('ACCIDENT FREE'), 'DESC-accifree'] = 1
    df['DESC-accifree'] = df['DESC-accifree'].fillna(0).astype(int)

    df.loc[df['description'].str.contains('WARRANTY'), 'DESC-warranty'] = 1
    df['DESC-warranty'] = df['DESC-warranty'].fillna(0).astype(int)

    df.loc[df['description'].str.contains('TRADE IN'), 'DESC-tradein'] = 1
    df['DESC-tradein'] = df['DESC-tradein'].fillna(0).astype(int)

    df = df.drop(columns = 'description')

    return df

def EXTRACT_accessories(df): 
    df['accessories'] = df['accessories'].astype(str).str.upper()

    df.loc[df['accessories'].str.contains('INSPECTION'), 'ACCESS-inspection'] = 1
    df['ACCESS-inspection'] = df['ACCESS-inspection'].fillna(0).astype(int)

    df.loc[df['accessories'].str.contains('AUDIO'), 'ACCESS-audio'] = 1
    df['ACCESS-audio'] = df['ACCESS-audio'].fillna(0).astype(int)

    df.loc[df['accessories'].str.contains('CAMERA'), 'ACCESS-camera'] = 1
    df['ACCESS-camera'] = df['ACCESS-camera'].fillna(0).astype(int)

    df.loc[(df['accessories'].str.contains('SPORT RIM')) | (df['accessories'].str.contains('SPORTS RIM')) , 'ACCESS-sportrim'] = 1
    df['ACCESS-sportrim'] = df['ACCESS-sportrim'].fillna(0).astype(int)
    
    df = df.drop(columns = 'accessories')

    return df



def CALC_currentage(df):
    
    df['AGE-currentMANU'] = datetime.datetime.now().year - df['manufactured'].astype(int)

    df['reg_date_temp']   = df['original_reg_date'].fillna(df['reg_date'])
    df['AGE-currentREG']  = df.apply(lambda x: dateutil.relativedelta.relativedelta(x['reg_date_temp'], datetime.datetime.now()).months, axis=1)
    df = df.drop(columns = ['reg_date_temp',])

    return(df)

def CALC_remainage(df):

    df['title'] = df['title'].str.upper()

    df.loc[df['title'].str.contains('COE'), 'coe_temp'] = df['title'].str.split(' ').str[-1]
    df['coe_temp1']         = df['coe_temp'].str.replace(')','')
    df['coe_temp1']         = pd.to_datetime(df['coe_temp1'], format = "%m/%Y",errors='coerce')
    df['coe_temp2']         = df[['original_reg_date', 'reg_date']].max(axis = 1) + pd.offsets.DateOffset(years=10)
    df['coe_enddate']       = df['coe_temp1'].fillna(df['coe_temp2'])
    df['AGE-remaining']     = df.apply(lambda x: dateutil.relativedelta.relativedelta(x['coe_enddate'], datetime.datetime.now()).years, axis=1)
    df                      = df.drop(columns = ["coe_temp", "coe_temp1", "coe_temp2", "coe_enddate"])

    return df

def IMPUTENULL_power(df):
    related_cols =  ['TRANSMISSION-manual',
                     'TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',]
    df_temp = df[related_cols + ['power']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['power'] = df['power'].fillna(df_temp[0])

    return df

def IMPUTENULL_curbweight(df):
    related_cols =  ['TRANSMISSION-manual',
                     'TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',]
    df_temp = df[related_cols + ['curb_weight']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['curb_weight'] = df['curb_weight'].fillna(df_temp[0])

    return df

def IMPUTENULL_enginecap(df):
    related_cols =  ['TRANSMISSION-manual',
                     'TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',]
    df_temp = df[related_cols + ['engine_cap']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['engine_cap'] = df['engine_cap'].fillna(df_temp[0])

    return df

def IMPUTENULL_depreciation(df):
    related_cols =  ['TRANSMISSION-manual',
                     'TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',
                     'AGE-currentMANU',            
                     'AGE-remaining'
                    ]
    df_temp = df[related_cols + ['depreciation']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['depreciation'] = df['depreciation'].fillna(df_temp[0])

    return df

def IMPUTENULL_arf(df):
    related_cols =  ['TRANSMISSION-manual',
                     'TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',
                     'AGE-currentMANU',
                     'AGE-remaining'
                    ]
    df_temp = df[related_cols + ['arf']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['arf'] = df['arf'].fillna(df_temp[0])

    return df

def IMPUTENULL_omv(df):
    related_cols =  ['TRANSMISSION-manual',
                     'TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',
                     'AGE-currentMANU',
                     'AGE-remaining'
                    ]
    df_temp = df[related_cols + ['omv']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['omv'] = df['omv'].fillna(df_temp[0])

    return df

def IMPUTENULL_mileage(df):
    related_cols =  ['TRANSMISSION-manual',
                     'AGE-currentMANU',
                     'AGE-remaining'
                    ]
    df_temp = df[related_cols + ['mileage']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['mileage'] = df['mileage'].fillna(df_temp[0])

    return df

def IMPUTENULL_roadtax(df):
    related_cols =  ['TYPE-binenc0',
                     'TYPE-binenc1',
                     'TYPE-binenc2',
                     'TYPE-binenc3',
                     'MAKE-binenc0',
                     'MAKE-binenc1',
                     'MAKE-binenc2',
                     'MAKE-binenc3',
                     'MAKE-binenc4',
                     'MAKE-binenc5',
                     'MAKE-binenc6',
                     'MODEL-binenc0',
                     'MODEL-binenc1',
                     'MODEL-binenc2',
                     'MODEL-binenc3',
                     'MODEL-binenc4',
                     'MODEL-binenc5',
                     'MODEL-binenc6',
                     'MODEL-binenc7',
                     'MODEL-binenc8',
                     'MODEL-binenc9',
                     'AGE-currentMANU',
                     'AGE-remaining'
                    ]
    df_temp = df[related_cols + ['road_tax']]
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_temp = imputer.fit_transform(df_temp)
    df_temp = pd.DataFrame(df_temp[:,-1])
    df['road_tax'] = df['road_tax'].fillna(df_temp[0])

    return df

def RATIO_ownertoage(df):
    df['OWNER-to-AGEcurrent'] = df['no_of_owners'] / df['AGE-currentMANU']
    df.loc[df['OWNER-to-AGEcurrent'] == np.inf, 'OWNER-to-AGEcurrent'] = 1
    return df

def RATIO_miletoage(df):
    df['MILEAGE-to-AGEcurrent'] = df['mileage'] / df['AGE-currentMANU']
    df.loc[df['MILEAGE-to-AGEcurrent'] == np.inf, 'MILEAGE-to-AGEcurrent'] = df['mileage']
    return df

def RATIO_miletoowner(df):
    df['MILEAGE-to-OWNER'] = df['mileage'] / df['no_of_owners']

    return df

def RATIO_curbtopower(df):
    df['CURB-to-POWER'] = df['curb_weight'] / df['power']
    df.loc[df['CURB-to-POWER'] == np.inf, 'CURB-to-POWER'] = df[~df['CURB-to-POWER'].isnull()]['CURB-to-POWER'].min()
    return(df)
    
def RATIO_indirectcost(df):    
    df['INDIRECTcost'] = df['AGE-remaining'] * (df['road_tax'] + df['depreciation']) - df['dereg_value']
    return(df)

def RATIO_ARFtoOMV(df):    
    df['ARF-to-OMV'] = df['arf'] / df['omv']
    return(df)

def RATIO_ARFtoDEREG(df):    
    df['ARF-to-DEREG'] = df['arf'] / df['dereg_value']
    df.loc[df['ARF-to-DEREG'] == np.inf, 'ARF-to-DEREG'] = 0
    
    return(df)


ENCODED_feature_list = ['curb_weight',
                        'power',
                        'engine_cap',
                        'no_of_owners',
                        'depreciation',
                        'coe',
                        'road_tax',
                        'dereg_value',
                        'mileage',
                        'omv',
                        'arf',
                        'opc_scheme',
                        
                        'CAT-electriccars',
                        'CAT-hybridcars',
                        'CAT-coecar', 
                        'CAT-parfcar', 
                        'CAT-lowmileagecar',
                        'CAT-directownersale',
                        'CAT-consignmentcar',     
                        'CAT-importedusedvehicle',
                        'CAT-premiumadcar',
                        'CAT-sgcarmartwarrantycars',
                        'CAT-staevaluatedcar',
                        'CAT-almostnewcar',
                        'CAT-rare&exotic',
                        'CAT-vintagecars',
                        
                        'TRANSMISSION-manual',
                        'TYPE-binenc0',
                        'TYPE-binenc1',
                        'TYPE-binenc2',
                        'TYPE-binenc3',
                        'MAKE-binenc0',
                        'MAKE-binenc1',
                        'MAKE-binenc2',
                        'MAKE-binenc3',
                        'MAKE-binenc4',
                        'MAKE-binenc5',
                        'MAKE-binenc6',
                        'FUEL-binenc0',
                        'FUEL-binenc1',
                        'FUEL-binenc2',
                        'MODEL-binenc0',
                        'MODEL-binenc1',
                        'MODEL-binenc2',
                        'MODEL-binenc3',
                        'MODEL-binenc4',
                        'MODEL-binenc5',
                        'MODEL-binenc6',
                        'MODEL-binenc7',
                        'MODEL-binenc8',
                        'MODEL-binenc9',
                        'FEAT-charged_engine',
                        'FEAT-srs_airbag',
                        'FEAT-keyless_ss',
                         'FEAT-guarantee',
                         'FEAT-awd',
                         'FEAT-4wd',
                         'FEAT-rwd',
                         'FEAT-fwd',
                         'DESC-loan',
                         'DESC-maintained',
                         'DESC-accifree',
                         'DESC-warranty',
                         'DESC-tradein',
                         'ACCESS-inspection',
                         'ACCESS-audio',
                         'ACCESS-camera',
                         'ACCESS-sportrim',
                        'AGE-currentMANU',
                        #'AGE-currentREG',
                        'AGE-remaining',
                        'OWNER-to-AGEcurrent',
                        'MILEAGE-to-AGEcurrent',
                        'MILEAGE-to-OWNER',
                        'CURB-to-POWER',
                        'INDIRECTcost',
                        'ARF-to-OMV',
                        'ARF-to-DEREG',]

CAT_feature_list = ['curb_weight',
                    'power',
                    'engine_cap',
                    'no_of_owners',
                    'depreciation',
                    'coe',
                    'road_tax',
                    'dereg_value',
                    'mileage',
                    'omv',
                    'arf',
                    'opc_scheme',
                    
                    'CAT-elechybrid',
                    'CAT-coeparf',
                    'CAT-lowmileagecar',
                    'CAT-directownersale',
                    'CAT-consignmentcar',     
                    'CAT-importedusedvehicle',
                    'CAT-premiumadcar',
                    'CAT-sgcarmartwarrantycars',
                    'CAT-staevaluatedcar',
                    'CAT-almostnewcar',
                    'CAT-rare&exotic',
                    'CAT-vintagecars',
                    
                    'TRANSMISSION-manual',
                    'make',
                    'type_of_vehicle',
                    'fuel_type',
                    'model',
                    
                    'FEAT-charged_engine',
                    'FEAT-srs_airbag',
                    'FEAT-keyless_ss',
                     'FEAT-guarantee',
                     'FEAT-awd',
                     'FEAT-4wd',
                     'FEAT-rwd',
                     'FEAT-fwd',
                     'DESC-loan',
                     'DESC-maintained',
                     'DESC-accifree',
                     'DESC-warranty',
                     'DESC-tradein',
                     'ACCESS-inspection',
                     'ACCESS-audio',
                     'ACCESS-camera',
                     'ACCESS-sportrim',
                    'AGE-currentMANU',
                    #'AGE-currentREG',
                    'AGE-remaining',
                    'OWNER-to-AGEcurrent',
                    'MILEAGE-to-AGEcurrent',
                    'MILEAGE-to-OWNER',
                    'CURB-to-POWER',
                    'INDIRECTcost',
                    'ARF-to-OMV',
                    'ARF-to-DEREG',]

target = 'price'

def model_preprocessing(input_df):
    
    input_df = input_df.drop(columns = ['eco_category', # same value for all records
                                        'indicative_price', # same value for all records
                                        'lifespan', #more than 90% of records null values 
                                       ])
    
    ####################### data processing ######################
    #####################################################################
    
    input_df = DEDUP_listingid(df = input_df)
    
    input_df = FIXNULL_make(df = input_df)
    input_df = FIXNULL_manufacture(df = input_df)
    input_df = FIXNULL_ownercount(df = input_df)
    input_df = FIXNULL_dereg(df = input_df)
    
    input_df = ENCODE_category(df = input_df, drop = False)
    input_df = ENCODE_transmission(df = input_df, drop=True)
    input_df = ENCODE_vehtype(df=input_df, drop=False)
    input_df = ENCODE_make(df=input_df, drop=False)
    input_df = ENCODE_fueltype(df=input_df, drop=False)
    input_df = ENCODE_model(df=input_df, drop=False)
    input_df = ENCODE_opcscheme(df=input_df)
    
    input_df = EXTRACT_features(df = input_df)
    input_df = EXTRACT_desc(df = input_df)
    input_df = EXTRACT_accessories(df = input_df)
    
    input_df = CALC_currentage(df = input_df)
    input_df = CALC_remainage(df = input_df)
    
    input_df = IMPUTENULL_power(df=input_df)
    input_df = IMPUTENULL_curbweight(df=input_df)
    input_df = IMPUTENULL_enginecap(df=input_df)
    input_df = IMPUTENULL_depreciation(df=input_df)
    input_df = IMPUTENULL_omv(df=input_df)
    input_df = IMPUTENULL_arf(df=input_df)
    input_df = IMPUTENULL_mileage(df=input_df)
    input_df = IMPUTENULL_roadtax(df=input_df)
    
    input_df = RATIO_ownertoage(df=input_df)
    input_df = RATIO_miletoage(df=input_df)
    input_df = RATIO_curbtopower(df=input_df)
    input_df = RATIO_indirectcost(df=input_df)
    input_df = RATIO_ARFtoOMV(df=input_df)
    input_df = RATIO_ARFtoDEREG(df=input_df)
    input_df = RATIO_miletoowner(df=input_df)

    cat_features = ["make", "type_of_vehicle", "fuel_type", "model", "CAT-elechybrid", "CAT-coeparf",]
    input_df[cat_features] = input_df[cat_features].astype('category')
    
    return input_df

In [5]:
train_df_RAW = pd.read_csv(r"./data/train.csv")
test_df_RAW = pd.read_csv(r"./data/test.csv")

####################### basic data definitions ######################
#####################################################################
# listing_id          - the unique id for the listing of a used car OK
# title               - title of the listing; free text attribute typically containing the make, model, engine type/size
# make                - make/brand of the car OK
# model               - model of the car (for the given make/brand) NOT SURE
# description         - title of the listing; free text attribute NOT SURE
# manufactured        - Year the car was manufactured OK
# original_reg_date   - Original registration data; date format string 
# reg_date            - Registration data; date format string
# type_of_vehicle     - Type of vehicle (e.g., "sports car", "luxury sedan")
# category            - Category of the car; list of categories, comma-separated
# transmission        - Type of transmission ("auto" or "manual")
# curb_weight         - Weight in kg of the vehicle without any passenger or items
# power               - Power of engine in kW
# fuel_type           - Fuel type (e.g, "petrol", "diesel", "electric")
# engine_cap          - Displacement of engine in cc (cubic centimeter)
# no_of_owners        - Number of previous owners (>=1 since all cars are used)
# depreciation        - Annual depreciation in SGD is the amount the owner loses on the value of the vehicle per year based on the assumption that the vehicle is deregistered only at the end of its 10-yr COE life-span
# coe                 - Certificate of entitlement value in SGD when first registered
# road_tax            - Road tax value in SGD calculated based on the engine capacity on a per annum basis
# dereg_value         - deregistration value in SGD one will get back from the government upon deregistering the vehicle for use in Singapore
# mileage             - Number of kilometers driven
# omv                 - Open Market Value in SGD assessed by the Singapore Customs
# arf                 - Additional Registration Fee in SGD is a tax imposed upon registration of a vehicle
# opc_scheme          - Off-peak car scheme
# lifespan            - Date stated on the lifespan marks the day that the vehicle must be deregistered
# eco_category        - Eco category of vehicle
# features            - Noteworthy features; free text attribute
# accessories         - Noteworthy accessories ; free text attribute
# indicative_price    - General guide to the price in SGD of the vehicle
# price               - Resale price in SGD of the car

train_df = model_preprocessing(input_df = train_df_RAW)
train_df_base, train_df_eval = train_test_split(train_df, test_size=0.2, random_state=888888)
test_df = model_preprocessing(input_df = test_df_RAW)

In [313]:
# def count_ngrams(series: pd.Series, n: int):
#     ngrams = series.copy().str.split(' ').explode()
#     for i in range(1, n):
#         ngrams += ' ' + ngrams.groupby(level=0).shift(-i)
#         ngrams = ngrams.dropna()
#     return ngrams.value_counts()  

In [21]:
####################### hyperparameters tuning ######################
#####################################################################

# Define the hyperparameter distributions
param_dist = {
            'learning_rate'     : stats.uniform(0.01, 0.1),
            'gamma'             : stats.uniform(0.01, 0.9),    
            'alpha'             : stats.uniform(0.01, 0.1),     
            
            'subsample'         : stats.uniform(0.3, 0.7),
            'colsample_bytree'  : stats.uniform(0.3, 0.7),
            'early_stopping_rounds' : stats.randint(8, 18), 
            'min_child_weight'  : stats.randint(3, 88),   
            }

xgb_model = xgboost.XGBRegressor(random_state = 888888, 
                                 n_estimators = 1888,
                                 max_depth    = 5,
                                 n_jobs       = -1, )

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(xgb_model, 
                                   param_distributions  = param_dist, 
                                   n_iter               = 88, 
                                   cv                   = 5, 
                                   random_state         = 888888,
                                   scoring              = 'neg_root_mean_squared_error', 
                                  )

# Fit the RandomizedSearchCV object to the training data
random_search.fit(train_df_base[ENCODED_feature_list], train_df_base[target],
                 eval_set=[(train_df_eval[ENCODED_feature_list], train_df_eval[target],)], verbose = False)
ENCODED_random_search_df = pd.DataFrame(random_search.cv_results_)
ENCODED_random_search_df = ENCODED_random_search_df[[ 'params',
                                     'split0_test_score',
                                     'split1_test_score',
                                     'split2_test_score',
                                     'split3_test_score',
                                     'split4_test_score',
                                     'mean_test_score',
                                     'std_test_score',
                                     'rank_test_score']]

ENCODED_random_search_df['objfunc'] = ENCODED_random_search_df[['split0_test_score',
                                                'split1_test_score',
                                                'split2_test_score',
                                                'split3_test_score',
                                                'split4_test_score',]].median(axis=1)
ENCODED_random_search_df = ENCODED_random_search_df.sort_values('objfunc', ascending = False)
ENCODED_random_search_df

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,objfunc
34,"{'alpha': 0.03679001717924471, 'colsample_bytr...",-24882.341459,-20992.742653,-26830.008247,-21359.886059,-20022.956162,-22817.586916,2593.362516,8,-21359.886059
76,"{'alpha': 0.07809528366287392, 'colsample_bytr...",-27178.187389,-21477.536368,-27199.003961,-20887.142372,-18127.867424,-22973.947503,3622.298419,13,-21477.536368
28,"{'alpha': 0.027840647836793533, 'colsample_byt...",-27696.135901,-21477.574695,-27627.108634,-21456.397668,-20048.665300,-23661.176440,3307.232038,30,-21477.574695
11,"{'alpha': 0.10696162820066175, 'colsample_bytr...",-27216.539449,-21650.323222,-27274.128319,-21502.056610,-19685.228326,-23465.655185,3162.795507,27,-21650.323222
24,"{'alpha': 0.07598328565775889, 'colsample_bytr...",-25371.701904,-21717.846694,-25892.380346,-21340.746437,-20207.881988,-22906.111474,2286.469820,10,-21717.846694
...,...,...,...,...,...,...,...,...,...,...
9,"{'alpha': 0.1068967351686765, 'colsample_bytre...",-31149.472464,-33185.515309,-37194.759977,-34879.005172,-28109.615255,-32903.673635,3113.037388,81,-33185.515309
29,"{'alpha': 0.06289360454715685, 'colsample_bytr...",-37250.454214,-35048.525136,-41012.660812,-43835.822026,-35323.696158,-38494.231669,3416.057499,87,-37250.454214
53,"{'alpha': 0.08128034722276746, 'colsample_bytr...",-37343.043021,-36050.917012,-39923.519351,-39210.018794,-31147.422423,-36734.984120,3109.697179,85,-37343.043021
40,"{'alpha': 0.08140978041922634, 'colsample_bytr...",-38190.070071,-36952.895168,-42041.782208,-40627.204318,-34144.725290,-38391.335411,2773.740694,86,-38190.070071


In [25]:
####################### model training (USING ENCODED FEATURES) ######################
#############################################################

best_params = ENCODED_random_search_df['params'].iloc[0]

ENCODED_xbg_model = xgboost.XGBRegressor(random_state           = 888888,
                                         n_estimators           = 1888,
                                         n_jobs                 = -1,
                                         max_depth              = 5,
                                         early_stopping_rounds  = best_params['early_stopping_rounds'],
                                         
                                         learning_rate          = best_params['learning_rate'], #  step size shrinkage used in updates to prevent overfitting. Lower values make the model more robust by taking smaller steps.
                                         gamma                  = best_params['gamma'], #minimum loss reduction required to make a further partition on a leaf node of the tree. Higher values increase the regularization.
                                         alpha                  = best_params['alpha'], #L2 regularization term on weights. 
                                         
                                         subsample              = best_params['subsample'], #percentage of rows used for each tree construction. 
                                         colsample_bytree       = best_params['colsample_bytree'], #percentage of columns used for each tree construction
                                         min_child_weight       = best_params['min_child_weight'], #Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, then the building process will give up further partitioning.
                                         verbosity = 2)

ENCODED_xbg_model.fit(train_df_base[ENCODED_feature_list], train_df_base[target],
                      eval_set=[(train_df_eval[ENCODED_feature_list], train_df_eval[target],)], verbose = False)
eval_rmse = mean_squared_error(train_df_eval[target], ENCODED_xbg_model.predict(train_df_eval[ENCODED_feature_list]), squared=False)
print(f"FINAL EVAL RMSE SCORE: {eval_rmse}")

submission_df = pd.DataFrame(ENCODED_xbg_model.predict(test_df[ENCODED_feature_list])).reset_index().rename(columns = {'index':"Id", 0:'Predicted'})
submission_df['Predicted'] = round(submission_df['Predicted'])
submission_df.to_csv(r"./data/submission.csv", index = False)

FINAL EVAL RMSE SCORE: 23958.532428656355


In [19]:
####################### hyperparameters tuning ######################
#####################################################################

# Define the hyperparameter distributions
param_dist = {
            'learning_rate'     : stats.uniform(0.01, 0.1),
            'gamma'             : stats.uniform(0.01, 0.9),    
            'alpha'             : stats.uniform(0.01, 0.1),     
            
            'subsample'         : stats.uniform(0.3, 0.7),
            'colsample_bytree'  : stats.uniform(0.3, 0.7),
            'early_stopping_rounds' : stats.randint(8, 18), 
            'min_child_weight'  : stats.randint(3, 88),   
            }

xgb_model = xgboost.XGBRegressor(random_state = 888888, 
                                 n_estimators = 1888,
                                 max_depth    = 5,              
                                 enable_categorical    = True,
                                 n_jobs = -1)

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(xgb_model, 
                                   param_distributions  = param_dist, 
                                   n_iter               = 88, 
                                   cv                   = 5, 
                                   random_state         = 888888,
                                   scoring              = 'neg_root_mean_squared_error',)

# Fit the RandomizedSearchCV object to the training data
random_search.fit(train_df_base[CAT_feature_list], train_df_base[target],
                  eval_set=[(train_df_eval[CAT_feature_list], train_df_eval[target],)], verbose = False)
CAT_random_search_df = pd.DataFrame(random_search.cv_results_)
CAT_random_search_df = CAT_random_search_df[['params',
                                     'split0_test_score',
                                     'split1_test_score',
                                     'split2_test_score',
                                     'split3_test_score',
                                     'split4_test_score',
                                     'mean_test_score',
                                     'std_test_score',
                                     'rank_test_score']]

CAT_random_search_df['objfunc'] = CAT_random_search_df[['split0_test_score',
                                                'split1_test_score',
                                                'split2_test_score',
                                                'split3_test_score',
                                                'split4_test_score',]].median(axis=1)
CAT_random_search_df = CAT_random_search_df.sort_values('objfunc', ascending = False)


In [20]:
####################### model training (USING CAT FEATURES) ######################
#############################################################

best_params = CAT_random_search_df['params'].iloc[0]


CAT_xbg_model = xgboost.XGBRegressor(random_state           = 888888,
                                     n_estimators           = 1888,
                                     enable_categorical     = True,
                                     n_jobs                 = -1,
                                     max_depth              = 5,
                                     early_stopping_rounds  = best_params['early_stopping_rounds'],
                                     
                                     learning_rate          = best_params['learning_rate'], #  step size shrinkage used in updates to prevent overfitting. Lower values make the model more robust by taking smaller steps.
                                     gamma                  = best_params['gamma'], #minimum loss reduction required to make a further partition on a leaf node of the tree. Higher values increase the regularization.
                                     alpha                  = best_params['alpha'], #L2 regularization term on weights. 
                                     
                                     subsample              = best_params['subsample'], #percentage of rows used for each tree construction. 
                                     colsample_bytree       = best_params['colsample_bytree'], #percentage of columns used for each tree construction
                                     min_child_weight       = best_params['min_child_weight'], #Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, then the building process will give up further partitioning.
                                     verbosity = 2)

CAT_xbg_model.fit(train_df_base[CAT_feature_list], train_df_base[target],
                      eval_set=[(train_df_eval[CAT_feature_list], train_df_eval[target],)], verbose = False)

eval_rmse = mean_squared_error(train_df_eval[target], CAT_xbg_model.predict(train_df_eval[CAT_feature_list]), squared=False)
print(f"FINAL EVAL RMSE SCORE: {eval_rmse}")

submission_df = pd.DataFrame(CAT_xbg_model.predict(test_df[CAT_feature_list])).reset_index().rename(columns = {'index':"Id", 0:'Predicted'})
submission_df['Predicted'] = round(submission_df['Predicted'])
submission_df.to_csv(r"./data/submission.csv", index = False)

FINAL EVAL RMSE SCORE: 25524.914110039103


In [ ]:
CAT_importance = pd.Series(CAT_xbg_model.feature_importances_, index = CAT_xbg_model.feature_names_in_).reset_index()
CAT_importance = CAT_importance.sort_values(0, ascending = False).reset_index(drop=True)
CAT_importance.head(50)